In [1]:
#!pip install tensorflow==2.17.1

In [2]:
import numpy as np
import tensorflow as tf

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

2024-11-29 20:09:44.958353: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-29 20:09:44.976616: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-29 20:09:44.997330: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-29 20:09:45.003385: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-29 20:09:45.018382: I tensorflow/core/platform/cpu_feature_guar

In [3]:
tf.__version__

'2.17.1'

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Question 1 & 2:

In [5]:
# Function to define model by adding new dense layer and dropout
def make_model():
    model = Sequential([Input(shape=(200, 200, 3)),
                        Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
                        MaxPooling2D(pool_size=(2, 2)),
                        Flatten(),
                        Dense(64, activation="relu"),
                        Dense(1, activation="sigmoid")]
                      )
    optimizer = SGD(learning_rate=0.002, momentum=0.8)
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [6]:
model = make_model()

I0000 00:00:1732910987.142114   14854 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1732910987.191586   14854 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1732910987.191845   14854 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1732910987.192936   14854 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [7]:
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 198, 198, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 99, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 313632)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │    20,072,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,073,473 (76.57 MB)

 Trainable params: 20,073,473 (76.57 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# Build image generator for training (takes preprocessing input function)
train_gen = ImageDataGenerator(rescale=1./255)

# Load in train dataset into train generator
train_ds = train_gen.flow_from_directory(directory='./data/train/', # Train images directory
                                         target_size=(200, 200), # resize images to train faster
                                         class_mode="binary",
                                         batch_size=20) # 20 images per batch

# Create image generator for validation
test_gen = ImageDataGenerator(rescale=1./255)

# Load in image for validation
test_ds = test_gen.flow_from_directory(directory='./data/test/', # Validation image directory
                                     target_size=(200, 200),
                                     class_mode="binary",
                                     batch_size=20) 

Found 800 images belonging to 2 classes.
Found 201 images belonging to 2 classes.


In [9]:
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=test_ds
)

/opt/saturncloud/envs/saturn/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


I0000 00:00:1732910989.356597   14899 service.cc:146] XLA service 0x7fb4b00035c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732910989.356638   14899 service.cc:154]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-11-29 20:09:49.382089: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-11-29 20:09:49.484341: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


 3/40 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - accuracy: 0.3583 - loss: 1.1448

I0000 00:00:1732910990.819379   14899 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


40/40 ━━━━━━━━━━━━━━━━━━━━ 9s 166ms/step - accuracy: 0.4866 - loss: 0.8238 - val_accuracy: 0.5323 - val_loss: 0.6806
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 136ms/step - accuracy: 0.5922 - loss: 0.6727 - val_accuracy: 0.6219 - val_loss: 0.6630
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 131ms/step - accuracy: 0.6241 - loss: 0.6482 - val_accuracy: 0.6318 - val_loss: 0.6341
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 135ms/step - accuracy: 0.6923 - loss: 0.5944 - val_accuracy: 0.5721 - val_loss: 0.6847
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 130ms/step - accuracy: 0.6663 - loss: 0.5947 - val_accuracy: 0.6368 - val_loss: 0.6246
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 132ms/step - accuracy: 0.6671 - loss: 0.5910 - val_accuracy: 0.6418 - val_loss: 0.6204
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 132ms/step - accuracy: 0.7184 - loss: 0.5517 - val_accuracy: 0.6020 - val_loss: 0.6354
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 129ms/step - accuracy: 0.7346 - loss: 0.5521 - val_accuracy: 0.6169 - val_

# Question3

In [10]:
np.median(history.history['accuracy']).round(3)

0.674

# Question 4

In [11]:
np.std(history.history['loss']).round(3)

0.062

# Question 5

In [12]:
# Build image generator for training (takes preprocessing input function)
train_gen = ImageDataGenerator(rescale=1./255,
                               rotation_range=50,
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               zoom_range=0.1,
                               horizontal_flip=True,
                               fill_mode='nearest')

# Load in train dataset into train generator
train_ds = train_gen.flow_from_directory(directory='./data/train/', # Train images directory
                                         target_size=(200, 200), # resize images to train faster
                                         class_mode="binary",
                                         batch_size=20) # 20 images per batch

# Create image generator for validation
test_gen = ImageDataGenerator(rescale=1./255)

# Load in image for validation
test_ds = test_gen.flow_from_directory(directory='./data/test/', # Validation image directory
                                     target_size=(200, 200),
                                     class_mode="binary",
                                     batch_size=20) 

Found 800 images belonging to 2 classes.
Found 201 images belonging to 2 classes.


In [13]:
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=test_ds
)

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step - accuracy: 0.6413 - loss: 0.6441

/opt/saturncloud/envs/saturn/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 283ms/step - accuracy: 0.6417 - loss: 0.6437 - val_accuracy: 0.6567 - val_loss: 0.5939
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 276ms/step - accuracy: 0.6513 - loss: 0.6177 - val_accuracy: 0.6866 - val_loss: 0.6042
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 270ms/step - accuracy: 0.6602 - loss: 0.6138 - val_accuracy: 0.6716 - val_loss: 0.5906
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 275ms/step - accuracy: 0.6866 - loss: 0.6047 - val_accuracy: 0.6617 - val_loss: 0.5822
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 277ms/step - accuracy: 0.6821 - loss: 0.5936 - val_accuracy: 0.7015 - val_loss: 0.5701
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 286ms/step - accuracy: 0.6895 - loss: 0.6204 - val_accuracy: 0.6517 - val_loss: 0.6268
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 279ms/step - accuracy: 0.6959 - loss: 0.5913 - val_accuracy: 0.7164 - val_loss: 0.5595
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 279ms/step - accuracy: 0.7048 - loss: 0.5852 - val_accuracy: 0.681

In [14]:
np.mean(history.history['val_loss']).round(3)

0.582

# Question 6

In [15]:
np.mean(history.history['val_accuracy'][5:]).round(3)

0.691